# Automated Solutions for the Challenges

The main goal of this notebook is to offer automatic solutions for the challenges listed in the "code_challenge.csv" file. These solutions will be generated automatically using five different programming languages, showcasing their versatility and applicability. Additionally, the notebook will leverage ChatGPT, an AI language model, to aid in the generation of these automatic solutions.

By combining the power of multiple programming languages and the capabilities of ChatGPT, the notebook aims to streamline the process of providing code solutions for a diverse range of challenges. 

Overall, the notebook seeks to demonstrate the potential of using AI in conjunction with various programming languages to automate code generation for a wide array of challenges, fostering efficiency and innovation in the programming community.

![Amidst the branches of the tree of knowledge, a python stands gracefully, savoring a cup of coffee](./code_illustration/python_java.jpg)

## Top Five Versatile Programming Languages

Here are the top 5 most versatile programming languages:

1. Python - Python is one of the most popular and versatile languages used today. It's easy to learn and can be used for a wide range of applications like web development, data analysis, artificial intelligence, scientific computing, and more.

2. JavaScript - JavaScript is the language of the web. It powers dynamic web pages and web applications and can also be used for game development, mobile apps, servers, IoT devices and more. 

3. Java - Java is a widely used object-oriented language suitable for a variety of applications like web servers, Big Data analysis, enterprise applications and Android app development. Its portability across platforms is a big plus.

4. C# - C# works across multiple platforms like Windows, mobile, and web. It is used for desktop apps, mobile apps, web apps, web services, games, VR, databases, and enterprise software. 

5. C++ - C++ is a high-performance language used to build systems, drivers, games engines, desktop apps and also has applications in embedded systems. It offers direct hardware manipulation and is highly efficient.

Other versatile options are C, Go, Swift, Kotlin, PHP, and Rust but Python, JavaScript, Java, C# and C++ cover the broadest range of application domains and are most widely used in the industry. Their popularity, vast ecosystems and support make them a great choice.

##  Loading, Preprocessing, and Displaying CSV Data

In [ ]:
import pandas as pd
from cryptography.fernet import Fernet
import os
import getpass
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
import os
import getpass
import base64
import requests
import openai
import time

In [ ]:
df = pd.read_csv('code_challenge.csv', delimiter=';', encoding='latin') 

# Drop index column
df = df.reset_index(drop=True) 

# Display dataframe again
display(df)

## Filtering for Beginner-Level Challenges

In [ ]:
beginner_df = df.loc[df['difficulty'] == 'Beginner']
len(beginner_df)

## Accessing the Encrypted File to Obtain the API Key

In [ ]:
%run "secure-api-key-encryption-and-retrieval.ipynb"
password_for_decryption = getpass.getpass("Enter your password to decrypt the API key: ")
api_key = get_decrypted_api_key(password_for_decryption)

## Generating Solutions for Code Challenges Using ChatGPT

In [ ]:
def generate_code(prompt, api_key, language):
    api_url = "https://api.openai.com/v1/chat/completions"

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    lang_map = {
        "python": "python programmer",
        "java": "java developer",
        "javascript": "javascript developer",
        "c++": "C++ developer",
        "c#": "C# developer"
    }

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": f"You are a {lang_map[language]}. Write a {language} program to create a {prompt}. Generate only the {language} code without any additional comments."},
            {"role": "user", "content": prompt}
        ]
    }

    response = requests.post(api_url, headers=headers, json=data)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None

def create_file(save_path, file_name, code_string, language):
    ext_map = {
        "python": ".py",
        "java": ".java",
        "javascript": ".js",
        "c++": ".cpp",
        "c#": ".cs"
    }

    try:
        file_path = os.path.join(save_path, "code_challenge_solution", language, file_name + ext_map[language])
        os.makedirs(os.path.dirname(file_path), exist_ok=True)

        with open(file_path, "w",  encoding="utf-8") as file:
            file.write(code_string)
        print(f"{language.capitalize()} file '{file_name + ext_map[language]}' created successfully.")
    except Exception as e:
        print(f"An error occurred while creating the file: {file_name + ext_map[language]}", str(e))

current_path = os.getcwd()
languages = ["python", "java", "javascript", "c++", "c#"]

for index, row in beginner_df.iterrows():
    id_val = row['id']
    name_val = row['name']
    description_val = row['description']
    difficulty_val = row['difficulty']
    
    code_challenge_row = name_val + ' ' + description_val

    ext_map = {
        "python": ".py",
        "java": ".java",
        "javascript": ".js",
        "c++": ".cpp",
        "c#": ".cs"
    }

    for lang in languages:
        file_name = f"{id_val} {name_val}"
        file_path = os.path.join(current_path, "code_challenge_solution", lang, file_name + ext_map[lang])

        if os.path.exists(file_path):
            print(f"{lang.capitalize()} file '{file_name + ext_map[lang]}' already exists.")
            continue

        code_string = generate_code(code_challenge_row, api_key, lang)
        create_file(current_path, file_name, code_string, lang)
        time.sleep(23)
